In [0]:
!apt-get -qq -y update
!curl https://colab.chainer.org/install | sh -
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg >/dev/null
!pip -q install chainerrl
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0   6074      0 --:--:-- --:--:-- --:--:--  6048
********************************************************************************
GPU is not enabled!
Open "Runtime" > "Change runtime type" and set "Hardware accelerator" to "GPU".
********************************************************************************


In [0]:
%cd /content/
!git clone https://github.com/susumuota/gym-modeestimation.git
%cd gym-modeestimation
!pip install -e .
%cd ..

# I dunno why but it works...
%cd /content/gym-modeestimation


In [0]:
import gym
import gym_modeestimation

def main():
    env = gym.make('ModeEstimationEPS00-v0')
    obs = env.reset()
    for i in range(40):
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        print(env.t, env.n0, action, obs, reward, done, info)
        if done:
            obs = env.reset()
    env.close()

if __name__ == '__main__':
    main()

In [0]:
import sys
import logging
import time

import gym
import numpy as np

import chainer
import chainer.functions as F
import chainer.links as L

import chainerrl
from chainerrl.agents import a2c
from chainerrl import experiments
from chainerrl import links
from chainerrl import misc
from chainerrl.optimizers.nonbias_weight_decay import NonbiasWeightDecay
from chainerrl import policies
from chainerrl import v_function
from chainerrl.policy import Policy

In [0]:
class A2CFFSoftmax(chainer.ChainList, a2c.A2CModel):
    """An example of A2C feedforward softmax policy."""

    def __init__(self, ndim_obs, n_actions, hidden_sizes=(64, 64)):
        self.pi = policies.SoftmaxPolicy(
            model=links.MLP(ndim_obs, n_actions, hidden_sizes))
        self.v = links.MLP(ndim_obs, 1, hidden_sizes=hidden_sizes)
        super().__init__(self.pi, self.v)

    def pi_and_v(self, state):
        return self.pi(state), self.v(state)


In [0]:
def make_reward_filtered(env, reward_filter):
    old_step = env.step

    def step(action):
        observation, reward, done, info = old_step(action)
        reward = reward_filter(observation, reward)
        return observation, reward, done, info

    env.step = step

In [0]:
def make_env(env_name, process_seed, test=False, reward_scale_factor=1.0):
    env = gym.make(env_name)
    # Use different random seeds for train and test envs
    #process_seed = int(process_seeds[process_idx])
    env_seed = 2 ** 32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    # Cast observations to float32 because our model uses float32
    env = chainerrl.wrappers.CastObservationToFloat32(env)
    # Scale rewards observed by agents
    if not test:
        misc.env_modifiers.make_reward_filtered(env, lambda x: x * reward_scale_factor)
        #make_reward_filtered(env, lambda obs, reward: obs[0])
    return env


In [0]:
def make_batch_env(num_envs, env_name, seed, test=False, reward_scale_factor=1.0):
    process_seeds = np.arange(num_envs) + seed * num_envs
    return chainerrl.envs.MultiprocessVectorEnv([(lambda: make_env(env_name, int(process_seeds[idx]), test, reward_scale_factor)) for idx in range(num_envs)])


In [0]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')
gym.logger.set_level(40)

#env_name = 'CartPole-v0'
#env_name = 'MountainCar-v0'
#env_name = 'Pendulum-v0'
env_name = 'ModeEstimationEPS00-v0'




In [27]:
sample_env = make_env(env_name, 0, test=False, reward_scale_factor=1.0)
#timestep_limit = sample_env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')
timestep_limit = 30
obs_space = sample_env.observation_space
action_space = sample_env.action_space
sample_env.close()

print(timestep_limit, obs_space, action_space)

30 Discrete(10) Discrete(11)


/usr/local/lib/python3.6/dist-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [31]:
arch = 'FFSoftmax'
#arch = 'Gaussian'
#arch = 'SimpleGaussian'

if arch == 'SimpleGaussian':
    model = A2CSimpleGaussian(obs_space.low.size, action_space.low.size)
if arch == 'Gaussian':
    model = A2CGaussian(obs_space.low.size, action_space.low.size)
elif arch == 'Beta':
    model = A2CBeta(obs_space.low.size, action_space.low.size, action_space.low[0], action_space.high[0])
elif arch == 'FFSoftmax':
    model = A2CFFSoftmax(obs_space.n, action_space.n)
elif arch == 'FFMellowmax':
    model = A2CFFMellowmax(obs_space.low.size, action_space.n)

model
    

In [32]:
lr = 7e-4
rmsprop_epsilon = 1e-5
alpha = 0.99
max_grad_norm = 0.5
weight_decay = 0.0

optimizer = chainer.optimizers.RMSprop(lr, eps=rmsprop_epsilon, alpha=alpha)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.GradientClipping(max_grad_norm))
if weight_decay > 0:
    optimizer.add_hook(NonbiasWeightDecay(weight_decay))

optimizer


In [36]:
num_envs = 8
gamma = 0.99
#gpu = 0 # -1 for CPU
gpu = -1 # -1 for CPU
update_steps = 5

agent = a2c.A2C(model, optimizer,
                    gamma=gamma,
                    gpu=gpu,
                    num_processes=num_envs,
                    update_steps=update_steps)

#load = 'exp1/20181210T043953.099247/5000192_finish'
load = ''

if load:
    agent.load(load)


agent


In [0]:
seed = 0
reward_scale_factor = 1e-2
outdir = 'modeest1'

steps         =  1 * 10 ** 3
log_interval  =   1 * 10 ** 3
eval_interval =  10 * 10 ** 3
eval_n_runs = 10

start = time.time()

outdir = experiments.prepare_output_dir({}, outdir)

experiments.train_agent_batch_with_evaluation(
    agent=agent,
    env=make_batch_env(num_envs, env_name, seed, test=False, reward_scale_factor=reward_scale_factor),
    eval_env=make_batch_env(num_envs, env_name, seed, test=True, reward_scale_factor=reward_scale_factor),
    steps=steps,
    log_interval=log_interval,
    eval_n_runs=eval_n_runs,
    eval_interval=eval_interval,
    outdir=outdir,
)

end = time.time() - start
print(end / 60.0)
